In [1]:
!pip install tf-nightly # Installing tf-nightly to try to resolve compatibility issues.
import numpy as np
import pandas as pd


from datetime import timedelta
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
seq_length = 14
dt=pd.read_csv('dt.csv')
values=dt['weighted_sentiment']
# Create sequences: for each sample, X contains previous 14 days and y is next day's value.
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

X, y = create_sequences(values, seq_length)

# Reshape X to have the shape (samples, time_steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

print("X shape:", X.shape, "y shape:", y.shape)

# ------------------------------
# Step 2: Build and Train the LSTM Model
# ------------------------------

model = Sequential([
    # LSTM layer with 50 units, dropout, and recurrent dropout to avoid overfitting.
    LSTM(50, activation='tanh', return_sequences=False,
         input_shape=(seq_length, 1),
         dropout=0.2,             # 20% dropout on inputs
         recurrent_dropout=0.2),  # 20% dropout on recurrent connections
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Early stopping to avoid overfitting.
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

history = model.fit(X, y, epochs=100, batch_size=16, verbose=1, callbacks=[early_stop])

X shape: (628, 14, 1) y shape: (628,)
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0055
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0041
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0038
Epoch 4/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0044
Epoch 5/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0042
Epoch 6/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037
Epoch 7/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0032
Epoch 8/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0035
Epoch 9/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0034
Epoch 10/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0035
Epoch 11/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0040
Epoch 12/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0028
Epoch 13/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0032
Epoch 14/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0041
Epoch 15/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0032
Epoch 16/100


In [2]:
model.save("lstm.h5")